In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
from details import folder

In [4]:
import os

In [5]:
data = pd.read_csv(r"\\vector.e3b.columbia.edu\e3b\Globus Database\UWIN Tagged Photos\batch_001\detections_to_update_2024-02-07.csv", index_col=0)

In [6]:
data.head(4)

,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals
1,VID34840-00000.jpg,2023-01-05 17:00:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN
2,VID34840-00001.jpg,2023-01-05 17:30:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN
3,VID34840-00002.jpg,2023-01-06 03:04:45,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN
4,VID34840-00003.jpg,2023-01-06 03:04:46,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN


In [7]:
data["is_present"] = data['species'].apply(lambda x: "not_present" if pd.isna(x) else "present")

In [9]:
directory = os.path.join(folder, r"batch_001")

In [10]:
datagen = ImageDataGenerator(rescale=1./255.)

In [11]:
dir_files = [i for i in os.listdir(directory) if i.endswith('.jpg')]

In [12]:
len(dir_files)

13019

In [13]:
data[data['photoName'].isin(dir_files)]

,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals,is_present
1,VID34840-00000.jpg,2023-01-05 17:00:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,not_present
2,VID34840-00001.jpg,2023-01-05 17:30:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,not_present
3,VID34840-00002.jpg,2023-01-06 03:04:45,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,present
4,VID34840-00003.jpg,2023-01-06 03:04:46,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,present
5,VID34840-00004.jpg,2023-01-06 03:04:47,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,present
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,VID31205-01142.jpg,2022-07-16 21:30:17,7,Bird,2,Brower Park,BrPk,-73.942589,40.673672,1,1,NaN,NaN,present
14996,VID31205-01143.jpg,2022-07-16 21:39:08,7,Bird,1,Brower Park,BrPk,-73.942589,40.673672,1,1,NaN,NaN,present
14997,VID31205-01144.jpg,2022-07-16 21:39:10,2278,NaN,1,Brower Park,BrPk,-73.942589,40.673672,1,1,NaN,NaN,not_present
14998,VID31205-01145.jpg,2022-07-16 21:54:16,7,Bird,1,Brower Park,BrPk,-73.942589,40.673672,1,1,NaN,NaN,present


In [14]:
data.photoName.isin(dir_files).sum()

14999

In [15]:
data.photoName.nunique()

13019

In [15]:
training_data = datagen.flow_from_dataframe(dataframe=data, 
                                            directory=, 
                                            x_col='photoName', 
                                            y_col='is_present', 
                                            target_size=(224,224), 
                                            class_mode='binary', 
                                            batch_size=8)

Found 14999 validated image filenames belonging to 2 classes.


In [16]:
from keras.applications import InceptionV3

In [27]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [28]:
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.16)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
out = tf.keras.layers.Dense(2, activation='softmax')(x)

In [29]:
base_model.trainable = False

In [30]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = out)

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adamax(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)         ┃ Output Shape      ┃    Param # ┃ Connected to       ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1        │ (None, 224, 224,  │          0 │ -                  │
│ (InputLayer)         │ 3)                │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ conv2d_94 (Conv2D)   │ (None, 111, 111,  │        864 │ input_layer_1[0][… │
│                      │ 32)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ batch_normalization… │ (None, 111, 111,  │         96 │ conv2d_94[0][0]    │
│ (BatchNormalization) │ 32)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ activation_94        │ (None, 111, 111,  │          0 │ batch_normalizati… │
│ (Activation)         │ 32)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ conv2d_95 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation_94[0][… │
│                      │ 32)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ batch_normalization… │ (None, 109, 109,  │         96 │ conv2d_95[0][0]    │
│ (BatchNormalization) │ 32)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ activation_95        │ (None, 109, 109,  │          0 │ batch_normalizati… │
│ (Activation)         │ 32)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ conv2d_96 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_95[0][… │
│                      │ 64)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ batch_normalization… │ (None, 109, 109,  │        192 │ conv2d_96[0][0]    │
│ (BatchNormalization) │ 64)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ activation_96        │ (None, 109, 109,  │          0 │ batch_normalizati… │
│ (Activation)         │ 64)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ max_pooling2d_4      │ (None, 54, 54,    │          0 │ activation_96[0][… │
│ (MaxPooling2D)       │ 64)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ conv2d_97 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d_4[0… │
│                      │ 80)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ batch_normalization… │ (None, 54, 54,    │        240 │ conv2d_97[0][0]    │
│ (BatchNormalization) │ 80)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ activation_97        │ (None, 54, 54,    │          0 │ batch_normalizati… │
│ (Activation)         │ 80)               │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ conv2d_98 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_97[0][… │
│                      │ 192)              │            │                    │
├──────────────────────┼───────────────────┼────────────┼────────────────────┤
│ batch_normalization… │ (None, 52, 52,    │        576 │ conv2d_98[0][0]    │
│ (BatchNormalization) │ 192)              │            │                    │
├──────────────────────┼───────────────────┼──────

 Total params: 28,356,770 (108.17 MB)

 Trainable params: 6,553,986 (25.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [34]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='./best_weights_acc_incepv31/best_weights_acc.weights.h5', monitor='accuracy', save_best_only=True, mode='max', save_weights_only=True)
]

In [35]:
model.fit(training_data, epochs=6, callbacks=callbacks)

Epoch 1/6


C:\Users\ab5640\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1512s 798ms/step - accuracy: 0.7054 - loss: 1.7507
Epoch 2/6
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1500s 797ms/step - accuracy: 0.7659 - loss: 0.4767
Epoch 3/6
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1480s 787ms/step - accuracy: 0.7835 - loss: 0.4139
Epoch 4/6
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1489s 791ms/step - accuracy: 0.7998 - loss: 0.3791
Epoch 5/6
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1494s 794ms/step - accuracy: 0.8191 - loss: 0.3353
Epoch 6/6
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1502s 798ms/step - accuracy: 0.8336 - loss: 0.3083
